## Final Project

I followed the model described in this website: 
    https://stackabuse.com/text-classification-with-python-and-scikit-learn/

It's not working super well. It defintiely needs refinement. We could go back into the dataset to get a better idea of what it contains.

In [37]:
import pandas as pd
import re
import nltk 
from sklearn.datasets import load_files
nltk.download('stopwords')
import pickle
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/alicekrupczak/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [38]:
orig_data = pd.read_csv("archive/presidential_speeches.csv")
orig_data.head()

,Date,President,Party,Speech Title,Summary,Transcript,URL
0,1789-04-30,George Washington,Unaffiliated,First Inaugural Address,Washington calls on Congress to avoid local an...,Fellow Citizens of the Senate and the House of...,https://millercenter.org/the-presidency/presid...
1,1789-10-03,George Washington,Unaffiliated,Thanksgiving Proclamation,"At the request of Congress, Washington establi...",Whereas it is the duty of all Nations to ackno...,https://millercenter.org/the-presidency/presid...
2,1790-01-08,George Washington,Unaffiliated,First Annual Message to Congress,"In a wide ranging speech, President Washington...",Fellow Citizens of the Senate and House of Rep...,https://millercenter.org/the-presidency/presid...
3,1790-12-08,George Washington,Unaffiliated,Second Annual Message to Congress,Washington focuses on commerce in his second a...,Fellow citizens of the Senate and House of Rep...,https://millercenter.org/the-presidency/presid...
4,1790-12-29,George Washington,Unaffiliated,Talk to the Chiefs and Counselors of the Senec...,The President reassures the Seneca Nation that...,"I the President of the United States, by my ow...",https://millercenter.org/the-presidency/presid...


In [39]:
data = orig_data.drop(columns = ['Date', 'Party', 'Speech Title', 'Summary', 'URL'])

In [40]:
data.head()

,President,Transcript
0,George Washington,Fellow Citizens of the Senate and the House of...
1,George Washington,Whereas it is the duty of all Nations to ackno...
2,George Washington,Fellow Citizens of the Senate and House of Rep...
3,George Washington,Fellow citizens of the Senate and House of Rep...
4,George Washington,"I the President of the United States, by my ow..."


In [41]:
# Split data into labels and transcripts.

labels = data['President']
transcripts = data['Transcript']

In [69]:
# Text preprocessing, taken from https://stackabuse.com/text-classification-with-python-and-scikit-learn/
# This could be refined more.

documents = []

stemmer = WordNetLemmatizer()

for sen in range(0, len(transcripts)):
    # Remove all the special characters
    document = re.sub(r'\W', ' ', str(transcripts[sen]))
    
    # remove all single characters
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
    
    # Remove single characters from the start
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 
    
    # Substituting multiple spaces with single space
    document = re.sub(r'\s+', ' ', document, flags=re.I)
    
    # Removing prefixed 'b'
    document = re.sub(r'^b\s+', '', document)
    
    # Converting to Lowercase
    document = document.lower()
    
    
    ## Could not get this part to work. 
    # Lemmatization
#     document = document.split()

#     document = [stemmer.lemmatize(word) for word in document]
#     document = ' '.join(document)
    
    documents.append(document)

In [58]:
# Term frequency = (Number of Occurrences of a word)/(Total words in the document)
# IDF(word) = Log((Total number of documents)/(Number of documents containing the word))

# Converts elements in the documents into features.

from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(max_features=10000, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))
X = vectorizer.fit_transform(documents).toarray()

In [60]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidfconverter = TfidfTransformer()
X = tfidfconverter.fit_transform(X).toarray()

In [61]:
#split data
from sklearn.model_selection import train_test_split
train_features, test_features, train_labels, test_labels = train_test_split(X, labels, test_size=0.2, random_state=0)

In [63]:
#This is the classifier they used in the example I was following. We can experiment with others. 

from sklearn.ensemble import RandomForestClassifier

In [65]:
classifier = RandomForestClassifier(n_estimators=1000, random_state=0)
classifier.fit(train_features, train_labels) 

RandomForestClassifier(n_estimators=1000, random_state=0)

In [66]:
pred_labels = classifier.predict(test_features)

In [68]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(test_labels, pred_labels))
print(classification_report(test_labels, pred_labels))
print(accuracy_score(test_labels, pred_labels))

[[2 0 0 ... 0 0 0]
 [0 7 0 ... 0 0 0]
 [0 0 7 ... 0 0 0]
 ...
 [0 0 0 ... 2 0 0]
 [0 1 0 ... 0 5 0]
 [0 0 0 ... 0 0 0]]
                       precision    recall  f1-score   support

       Andrew Jackson       0.33      0.50      0.40         4
       Andrew Johnson       0.78      0.78      0.78         9
         Barack Obama       0.88      0.64      0.74        11
    Benjamin Harrison       0.67      0.33      0.44         6
         Bill Clinton       1.00      0.67      0.80         9
      Calvin Coolidge       1.00      0.67      0.80         3
    Chester A. Arthur       0.00      0.00      0.00         4
         Donald Trump       1.00      0.75      0.86         4
Franklin D. Roosevelt       0.42      1.00      0.59         5
      Franklin Pierce       1.00      0.50      0.67         2
    George H. W. Bush       1.00      0.20      0.33         5
       George W. Bush       0.56      1.00      0.71         5
    George Washington       1.00      1.00      1.00        

/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
